In [2]:
import pandas as pd
import h2o
import nltk
import numpy as np
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 16.0.1+9-24, mixed mode, sharing)
  Starting server from C:\Users\siraj\AppData\Roaming\Python\Python39\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\siraj\AppData\Local\Temp\tmpn86pl0jb
  JVM stdout: C:\Users\siraj\AppData\Local\Temp\tmpn86pl0jb\h2o_siraj_started_from_python.out
  JVM stderr: C:\Users\siraj\AppData\Local\Temp\tmpn86pl0jb\h2o_siraj_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Rome
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_siraj_rvzhm6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.947 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
dataset = pd.read_csv("Dataset_Smart.csv")

In [4]:
dataset

,Entity,Timestamp,Class
0,Imran Jami,01/03/2019 9:39,1
1,Siraj Munir,01/03/2019 9:36,1
2,Shaukat Wasi,01/03/2019 9:53,1
3,Zubair Ahmed Sheikh,01/03/2019 10:40,1
4,Asim Wagan,01/03/2019 11:15,1
...,...,...,...
4494,Abdullah Qamar,27/06/2022 23:36,2
4495,Siraj Munir,18/06/2022 14:53,2
4496,Shaukat Wasi,16/02/2022 23:26,2
4497,M. Jahanzaib,07/04/2022 18:02,2


In [5]:
Facts = dataset.Entity
Relation = dataset.Timestamp
Classification = dataset.Class

In [6]:
h2o_df=h2o.H2OFrame(np.hstack((Facts[:,None],Relation[:,None],Classification[:,None])),column_names=['Facts','Relation','Class'],column_types=['string','string','string'])

C:\Users\siraj\AppData\Local\Temp\ipykernel_1548\1764106392.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  h2o_df=h2o.H2OFrame(np.hstack((Facts[:,None],Relation[:,None],Classification[:,None])),column_names=['Facts','Relation','Class'],column_types=['string','string','string'])


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
h2o_df

Facts,Relation,Class
Imran Jami,01/03/2019 9:39,1
Siraj Munir,01/03/2019 9:36,1
Shaukat Wasi,01/03/2019 9:53,1
Zubair Ahmed Sheikh,01/03/2019 10:40,1
Asim Wagan,01/03/2019 11:15,1
Tafseer Ahmed,01/03/2019 9:36,1
Yaseen Khan,01/03/2019 9:20,1
M. Jahanzaib,01/03/2019 10:31,1
Hamza Gaya,01/03/2019 9:27,1
Abdul Rasheed,01/03/2019 11:01,1


In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [9]:
def tokenize(line):
    tokenized = line.tokenize("\\W+")
    tokenized = tokenized.tolower()
    tokenized = tokenized[(tokenized.nchar() >= 2) | (tokenized.isna()),:]
    tokenized_filter = tokenized[(tokenized.isna()) | (~ tokenized.isin(stop_words)),:]
    return tokenized_filter

In [10]:
final_data = tokenize(h2o_df["Facts"])
final_data_2 = tokenize(h2o_df["Relation"])

In [11]:
final_data.head()

C1
imran
jami
""
siraj
munir
""
shaukat
wasi
""
zubair


In [12]:
from h2o.estimators.word2vec import H2OWord2vecEstimator

In [13]:
vec_model = H2OWord2vecEstimator(vec_size = 100, model_id = "Vector_Model.model")
vec_model.train(training_frame=final_data)
vec_model.train(training_frame=final_data_2)

word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%
word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OWord2vecEstimator : Word2Vec
Model Key: Vector_Model.model

No summary for this model

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [14]:
h2o.save_model(vec_model,path='./')

'C:\\Users\\siraj\\Desktop\\Workout\\Deep-workout\\Final work for vectorization pipeline\\Vector_Model.model'

In [15]:
data_vecs_facts = vec_model.transform(final_data, aggregate_method = "AVERAGE")

In [24]:
data_vecs_facts.shape

(4499, 100)

In [25]:
data_vecs_facts.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,

In [26]:
data_vecs_reasoning = vec_model.transform(final_data_2, aggregate_method = "AVERAGE")

In [27]:
data_vecs_reasoning.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.0857271,-0.0317348,-0.0896656,0.00209449,0.0474614,0.0585676,0.0376502,0.0134088,0.0943641,-0.0931881,0.163682,0.0934911,-0.00671406,0.141949,0.0397847,-0.0891309,-0.0225342,0.0724394,-0.050613,0.150308,-0.156265,0.109172,0.0862037,0.0396831,0.0961583,-0.0189355,-0.0284714,0.118792,0.0492612,-0.0398628,-0.025871,-0.115144,0.0574241,0.0717864,0.0879977,-0.148054,-0.0463274,-0.25601,-0.0323847,-0.0785178,0.103336,0.0599663,0.183412,-0.0117705,-0.018594,0.0145209,0.175655,0.0883314,-0.15482,-0.0242695,0.0103799,-0.00291104,-0.0377698,0.0307057,0.0693746,-0.104265,0.00410365,-0.00217499,-0.0143195,0.117469,0.0548409,0.102495,-0.00406422,-0.12678,-0.0818817,0.104132,0.105681,-0.0207003,-0.0448866,0.125403,-0.109428,-0.0442235,0.105425,-0.0188309,0.00127585,-0.0290136,-0.110018,-0.0218542,0.0582644,-0.017732,-0.0522206,0.139584,-0.12483,-0.025701,0.0304386,-0.0228094,0.0436634,0.0418764,-0.0979344,-0.00366503,0.137459,-0.0274307,-0.0373195,-0.12455,-0.0281949,0.0135229,-0.163092,0.0538548,0.000491962,-0.107132
0.0912514,-0.0340899,-0.0986437,0.00109258,0.0521002,0.0640701,0.0388142,0.0132414,0.10219,-0.101542,0.178699,0.101704,-0.00859633,0.155557,0.0428505,-0.0966418,-0.0227706,0.0795554,-0.0558451,0.164632,-0.171555,0.118772,0.0928341,0.0424238,0.102278,-0.0192738,-0.0324106,0.128819,0.053262,-0.0439088,-0.0264247,-0.124874,0.0593485,0.0765234,0.0951078,-0.15908,-0.0505816,-0.281071,-0.0354835,-0.0849791,0.113569,0.0662877,0.199449,-0.0143615,-0.0178068,0.0140141,0.192893,0.0940032,-0.169105,-0.0266169,0.0116613,-0.00175012,-0.0417718,0.034816,0.0723049,-0.11295,0.00297945,-0.00097621,-0.0167309,0.128504,0.0576259,0.110771,-0.0051479,-0.135984,-0.0892781,0.111873,0.114597,-0.0244482,-0.0493224,0.137259,-0.117647,-0.0477248,0.113723,-0.0201253,0.00131295,-0.0312215,-0.120898,-0.0226076,0.0647191,-0.0216256,-0.0595036,0.152223,-0.135533,-0.0260174,0.033286,-0.026697,0.0482122,0.0453179,-0.105409,-0.00404627,0.148163,-0.0291052,-0.0409844,-0.13521,-0.0303938,0.0159788,-0.17861,0.0579768,0.00136882,-0.118652
0.0867189,-0.031513,-0.0916596,0.00413268,0.0505508,0.0591167,0.0395866,0.0123772,0.0941488,-0.0942666,0.165541,0.0967634,-0.00790745,0.14453,0.0403481,-0.092461,-0.0218349,0.0749257,-0.0520844,0.153844,-0.160468,0.111808,0.0863983,0.0400343,0.0963067,-0.0190764,-0.0283277,0.120586,0.0495633,-0.0411516,-0.0240302,-0.116364,0.0584389,0.0710411,0.0884544,-0.148272,-0.0472415,-0.261902,-0.0335978,-0.0802338,0.107878,0.0620487,0.187686,-0.0132097,-0.015975,0.0129794,0.178152,0.0888878,-0.158188,-0.0235807,0.0106618,-0.00363892,-0.0388592,0.0330755,0.0699582,-0.105422,0.00314131,-0.00211622,-0.0151652,0.120972,0.0539117,0.104837,-0.00318878,-0.127842,-0.0825222,0.107044,0.108129,-0.0196672,-0.0472866,0.129238,-0.112173,-0.0434499,0.10717,-0.0195172,0.00291804,-0.0294745,-0.11298,-0.0231063,0.0588106,-0.0191274,-0.0552398,0.141951,-0.12651,-0.0256855,0.030606,-0.0224896,0.0459409,0.0426711,-0.0971267,-0.00489118,0.140627,-0.0292705,-0.0388777,-0.125228,-0.0292083,0.0153113,-0.166996,0.053028,0.00158868,-0.109717
0.080353,-0.0312141,-0.0864273,-0.00191472,0.0499201,0.0576896,0.0339973,0.0133872,0.0925844,-0.0910491,0.158938,0.0939871,-0.00870341,0.13982,0.0347257,-0.0873256,-0.0239756,0.0714782,-0.0462881,0.145156,-0.155145,0.107007,0.085311,0.0334322,0.0901207,-0.0181617,-0.0304413,0.113517,0.0497522,-0.0377604,-0.0268118,-0.106614,0.0477314,0.0696372,0.0847396,-0.141035,-0.0424814,-0.250768,-0.0304431,-0.0749796,0.0959356,0.0582693,0.181995,-0.0151314,-0.0184988,0.00899725,0.174533,0.0844484,-0.150842,-0.0252079,0.00943749,-0.00127806,-0.0424072,0.0

In [31]:
# h2o_df_ext=h2o_df.cbind(data_vecs_facts)
h2o_df_ext = h2o_df.cbind(data_vecs_reasoning)

In [32]:
h2o_df_ext.head()

Facts,Relation,Class,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
Imran Jami,01/03/2019 9:39,1,0.0857271,-0.0317348,-0.0896656,0.00209449,0.0474614,0.0585676,0.0376502,0.0134088,0.0943641,-0.0931881,0.163682,0.0934911,-0.00671406,0.141949,0.0397847,-0.0891309,-0.0225342,0.0724394,-0.050613,0.150308,-0.156265,0.109172,0.0862037,0.0396831,0.0961583,-0.0189355,-0.0284714,0.118792,0.0492612,-0.0398628,-0.025871,-0.115144,0.0574241,0.0717864,0.0879977,-0.148054,-0.0463274,-0.25601,-0.0323847,-0.0785178,0.103336,0.0599663,0.183412,-0.0117705,-0.018594,0.0145209,0.175655,0.0883314,-0.15482,-0.0242695,0.0103799,-0.00291104,-0.0377698,0.0307057,0.0693746,-0.104265,0.00410365,-0.00217499,-0.0143195,0.117469,0.0548409,0.102495,-0.00406422,-0.12678,-0.0818817,0.104132,0.105681,-0.0207003,-0.0448866,0.125403,-0.109428,-0.0442235,0.105425,-0.0188309,0.00127585,-0.0290136,-0.110018,-0.0218542,0.0582644,-0.017732,-0.0522206,0.139584,-0.12483,-0.025701,0.0304386,-0.0228094,0.0436634,0.0418764,-0.0979344,-0.00366503,0.137459,-0.0274307,-0.0373195,-0.12455,-0.0281949,0.0135229,-0.163092,0.0538548,0.000491962,-0.107132
Siraj Munir,01/03/2019 9:36,1,0.0912514,-0.0340899,-0.0986437,0.00109258,0.0521002,0.0640701,0.0388142,0.0132414,0.10219,-0.101542,0.178699,0.101704,-0.00859633,0.155557,0.0428505,-0.0966418,-0.0227706,0.0795554,-0.0558451,0.164632,-0.171555,0.118772,0.0928341,0.0424238,0.102278,-0.0192738,-0.0324106,0.128819,0.053262,-0.0439088,-0.0264247,-0.124874,0.0593485,0.0765234,0.0951078,-0.15908,-0.0505816,-0.281071,-0.0354835,-0.0849791,0.113569,0.0662877,0.199449,-0.0143615,-0.0178068,0.0140141,0.192893,0.0940032,-0.169105,-0.0266169,0.0116613,-0.00175012,-0.0417718,0.034816,0.0723049,-0.11295,0.00297945,-0.00097621,-0.0167309,0.128504,0.0576259,0.110771,-0.0051479,-0.135984,-0.0892781,0.111873,0.114597,-0.0244482,-0.0493224,0.137259,-0.117647,-0.0477248,0.113723,-0.0201253,0.00131295,-0.0312215,-0.120898,-0.0226076,0.0647191,-0.0216256,-0.0595036,0.152223,-0.135533,-0.0260174,0.033286,-0.026697,0.0482122,0.0453179,-0.105409,-0.00404627,0.148163,-0.0291052,-0.0409844,-0.13521,-0.0303938,0.0159788,-0.17861,0.0579768,0.00136882,-0.118652
Shaukat Wasi,01/03/2019 9:53,1,0.0867189,-0.031513,-0.0916596,0.00413268,0.0505508,0.0591167,0.0395866,0.0123772,0.0941488,-0.0942666,0.165541,0.0967634,-0.00790745,0.14453,0.0403481,-0.092461,-0.0218349,0.0749257,-0.0520844,0.153844,-0.160468,0.111808,0.0863983,0.0400343,0.0963067,-0.0190764,-0.0283277,0.120586,0.0495633,-0.0411516,-0.0240302,-0.116364,0.0584389,0.0710411,0.0884544,-0.148272,-0.0472415,-0.261902,-0.0335978,-0.0802338,0.107878,0.0620487,0.187686,-0.0132097,-0.015975,0.0129794,0.178152,0.0888878,-0.158188,-0.0235807,0.0106618,-0.00363892,-0.0388592,0.0330755,0.0699582,-0.105422,0.00314131,-0.00211622,-0.0151652,0.120972,0.0539117,0.104837,-0.00318878,-0.127842,-0.0825222,0.107044,0.108129,-0.0196672,-0.0472866,0.129238,-0.112173,-0.0434499,0.10717,-0.0195172,0.00291804,-0.0294745,-0.11298,-0.0231063,0.0588106,-0.0191274,-0.0552398,0.141951,-0.12651,-0.0256855,0.030606,-0.0224896,0.0459409,0.0426711,-0.0971267,-0.00489118,0.140627,-0.0292705,-0.0388777,-0.125228,-0.0292083,0.0153113,-0.166996,0.053028,0.00158868,-0.109717
Zubair Ahmed Sheikh,01/03/2019 10:40,1,0.080353,-0.0312141,-0.0864273,-0.00191472,0.0499201,0.0576896,0.0339973,0.0133872,0.0925844,-0.0910491,0.158938,0.0939871,-0.00870341,0.13982,0.0347257,-0.0873256,-0.0239756,0.0714782,-0.0462881,0.145156,-0.155145,0.107007,0.085311,0.0334322,0.0901207,-0.0181617,-0.0304413,0.113517,0.0497522,-0.0377604,-0.0268118,-0.106614,0.0477314,0.0696372,0.0847396,-0.141035,-0.0424814,-0.250768,-0.0304431,

In [33]:
vectorzied_dataset = h2o.as_list(h2o_df_ext, use_pandas=True)

In [34]:
vectorzied_dataset.to_csv("Vectorized_Dataset.csv")